In [ ]:
import time, math, cmath, os
import numpy as np
import scipy as sp
from scipy.linalg import expm,sqrtm
import qiskit as qs
from qiskit.quantum_info import random_unitary, random_clifford, random_hermitian
from qiskit.opflow import I, X, Y, Z
from qiskit.quantum_info import Pauli
from tqdm import tqdm

# Parameters
global tol

In [ ]:
# 通过给定的H计算出相应N^{-1} channel的(X^D)^{-1}
def VX_calculate(H):
    D = len(H)
    n = int(math.log(len(H), 2))

    # diagonalize the Hamiltonian H
    l, V = np.linalg.eig(H)
    V_dag = V.conj().T

    # X matrix is (2^n, 2^n) , shrinked version of the original matrix
    # X_{j,i} = OriginalX_{ij,ij}
    X_mat = np.zeros((D, D), dtype=complex)
    for i in range(D):
        for j in range(D):
            for b in range(D):
                X_mat[j, i] += V_dag[i, b]*V_dag[j, b]*V[b, i]*V[b, j]
    X_inv = np.linalg.inv(X_mat)
    return l, V, X_mat, X_inv

def Direct_calculate_X(V):
    dim = len(V)
    V_dag = V.conj().T

    # X matrix is (2^n, 2^n) , shrinked version of the original matrix
    # X_{j,i} = OriginalX_{ij,ij}
    X_mat = np.zeros((dim, dim), dtype=complex)
    for i in range(dim):
        for j in range(dim):
            for b in range(dim):
                X_mat[j, i] += V_dag[i, b]*V_dag[j, b]*V[b, i]*V[b, j]
    X_inv = np.linalg.inv(X_mat)
    return X_mat, X_inv


In [ ]:
def I_str(n):
    res = I
    for i in range(n-1):
        res = res ^ I
    return res

def V_matrix(C, x_lst):
    n = len(x_lst)
    res = [[0 for i in range(n)] for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            res[i][j] = C/(x_lst[i]-x_lst[j])**6
            res[j][i] = C/(x_lst[i]-x_lst[j])**6
    return res

def Rydberg_Hamiltonian(Omega_lst, phi_lst, Delta_lst, v_matrix):
    V = v_matrix
    n = len(Omega_lst)
    # print('n=',n)
    X_str_lst = [X ^ I_str(n-1)]
    Y_str_lst = [Y ^ I_str(n-1)]
    Z_str_lst = [Z ^ I_str(n-1)]
    for i in range(n-2):
        X_str_lst.append(I_str(i+1) ^ X ^ I_str(n-i-2))
        Y_str_lst.append(I_str(i+1) ^ Y ^ I_str(n-i-2))
        Z_str_lst.append(I_str(i+1) ^ Z ^ I_str(n-i-2))
    X_str_lst.append(I_str(n-1) ^ X)
    Y_str_lst.append(I_str(n-1) ^ Y)
    Z_str_lst.append(I_str(n-1) ^ Z)

    res = Omega_lst[0]/2*np.cos(phi_lst[0]) * X_str_lst[0] - Omega_lst[0]/2*np.sin(
        phi_lst[0]) * Y_str_lst[0] - Delta_lst[0]/2 * Z_str_lst[0]
    for i in range(n-1):
        res = res + Omega_lst[i+1]/2*np.cos(phi_lst[i+1]) * X_str_lst[i+1] - Omega_lst[i+1]/2*np.sin(
            phi_lst[i+1]) * Y_str_lst[i+1] - Delta_lst[i+1]/2 * Z_str_lst[i+1]

    for i in range(n):
        for j in range(i+1, n):
            # print(i,j)
            Vij = V[i][j]/4
            if i == 0:
                if j == (j == i+1) & (j != n-1):
                    res = res + (Vij * (Z + I) ^ (Z + I) ^ I_str(n-2))
                elif j == n-1:
                    res = res + (Vij * (Z + I) ^ I_str(n-2) ^ (Z + I))
                else:
                    res = res + (Vij * (Z + I) ^ I_str(j-i-1)
                                 ^ (Z + I) ^ I_str(n-j-1))
            elif i == n-2:
                res = res + (Vij * I_str(n-2) ^ (Z + I) ^ (Z + I))
            else:
                if j == i+1:
                    #print((Vij* I_str(i) ^ (Z + I) ^ (Z + I) ^ I_str(n-j-1)))
                    res = res + (Vij * I_str(i) ^ (Z + I)
                                 ^ (Z + I) ^ I_str(n-j-1))
                elif j == n-1:
                    res = res + (Vij * I_str(i) ^ (Z + I)
                                 ^ I_str(j-i-1) ^ (Z + I))
                else:
                    res = res + (Vij * I_str(i) ^ (Z + I) ^
                                 I_str(j-i-1) ^ (Z + I) ^ I_str(n-j-1))

    return res.to_matrix()

In [ ]:
def H_map(Post_state, V, X_mat, X_inv):

    D = len(Post_state)
    V_dag = V.conj().T
    # get the input state for channel N^{-1}
    P = np.dot(V_dag, np.dot(Post_state, V))

    rho_hat = P / X_mat
    rho_hat_diag = np.dot(X_inv,np.diag(P))
    diag = np.arange(D)
    rho_hat[diag,diag] = rho_hat_diag

    rho_hat = np.dot(V, np.dot(rho_hat, V_dag))
    return rho_hat

In [ ]:
# Variance approximation
def Var_approx(obs,V,X_mat,M):
    dim = len(V)
    n = int(math.log(dim,2))

    V_dag = V.conj().T
    obs_spin = np.dot(V_dag,np.dot(obs,V))

    var = 0
    for i in range(dim):
        for j in range(dim):
            var += ((obs_spin[i][j]*obs_spin[j][i])/(X_mat[i][j]))
    var = var.real / dim
    var = var / M
    return var

# Variance ideal calculation
def Var_ideal(obs,rho,V,X_mat,X_inv,M):
    dim = len(V)
    n = int(math.log(dim,2))

    V_dag = V.conj().T
    obs_out = H_map(obs,V,X_mat,X_inv)
    obs_post = np.dot(V_dag,np.dot(obs_out,V))
    rho_post = np.dot(V_dag,np.dot(rho,V))

    var = 0
    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                # 计算 Y(i,j,k)
                y = 0
                for b in range(dim):
                    y += V[b][i]*V_dag[i][b] * V[b][j]*V_dag[j][b] * V[b][k]*V_dag[k][b]

                # 根据 i,j,k 有几个重复index分类
                if ((i==j) and (j==k)):
                    var += y*( obs_post[i][i]*obs_post[i][i]*rho_post[i][i] )
                elif i==j:
                    var += y*( obs_post[i][i]*obs_post[i][i]*rho_post[k][k] + 2*obs_post[i][k]*obs_post[i][i]*rho_post[k][i])
                elif j==k:
                    var += y*( obs_post[i][i]*obs_post[j][j]*rho_post[j][j] + obs_post[i][j]*obs_post[j][i]*rho_post[j][j] + obs_post[i][j]*obs_post[j][j]*rho_post[j][i] )
                elif k==i:
                    var += y*( obs_post[i][i]*obs_post[j][j]*rho_post[i][i] + obs_post[i][j]*obs_post[j][i]*rho_post[i][i] + obs_post[i][i]*obs_post[j][i]*rho_post[i][j] )
                else:
                    var += y*( obs_post[i][i]*obs_post[j][j]*rho_post[k][k] + obs_post[i][i]*obs_post[j][k]*rho_post[k][j] + obs_post[i][j]*obs_post[j][i]*rho_post[k][k] + obs_post[i][j]*obs_post[j][k]*rho_post[k][i] + obs_post[i][k]*obs_post[j][i]*rho_post[k][j] + obs_post[i][k]*obs_post[j][j]*rho_post[k][i] )

    var = var - (np.trace(np.dot(rho,obs)))**2

    var = var.real
    var = var / M
    return var

# Variance of global shadow
def Var_global(obs,rho,M):
    dim = len(rho)
    part1 = np.trace(np.dot(obs,obs)) + 2*np.trace(np.dot(np.dot(obs,obs),rho))
    part2 = 2*np.trace(obs)*np.trace(np.dot(obs,rho)) + (np.trace(obs))**2
    part3 = (np.trace(np.dot(obs,rho)))**2
    var = ((dim+1)*part1)/(dim+2) - part2/(dim+2) - part3
    var = var.real / M
    return var

In [ ]:

# ensemble 0: Wrong shadow which is a hybird of Hamiltonian experiments and Global post-processing
def Wrong_Global_Shadow(rho, l, V, t_min, t_max):
    dim = len(rho)
    t = t_min+(t_max-t_min)*np.random.rand()
    # U = e^(-iHt)
    V_dag = V.conj().T
    ll = [cmath.exp(-1j*t*l[i]) for i in range(dim)]
    L = np.diag(ll)
    U = np.dot(V, np.dot(L, V_dag))
    U_dag = U.conj().T

    evoluted_rho = np.dot(U, np.dot(rho, U_dag))
    prob_lst = [0 for i in range(dim)]
    for i in range(dim):
        if abs(evoluted_rho[i][i]) > 10**(tol):
            prob_lst[i] = evoluted_rho[i][i].real
        else:
            prob_lst[i] = 0
    prob_lst = prob_lst / sum(prob_lst)
    measurement_res = np.random.choice(np.arange(0, dim), p=prob_lst)

    # calculate the post-measurement state
    rho_hat = np.outer(U[measurement_res,:].conj(),U[measurement_res,:])
    rho_hat = (dim+1)*rho_hat - np.identity(dim)
    return rho_hat

# ensemble 1: Our shadow using Hamiltonian
def Hamiltonian_Shadow(rho, l, V, X_mat, X_inv, t_min, t_max):
    dim = len(rho)
    t = t_min+(t_max-t_min)*np.random.rand()
    # U = e^(-iHt)
    V_dag = V.conj().T
    ll = [cmath.exp(-1j*t*l[i]) for i in range(dim)]
    L = np.diag(ll)
    U = np.dot(V, np.dot(L, V_dag))
    U_dag = U.conj().T

    evoluted_rho = np.dot(U, np.dot(rho, U_dag))
    prob_lst = [0 for i in range(dim)]
    for i in range(dim):
        if abs(evoluted_rho[i][i]) > 10**(tol):
            prob_lst[i] = evoluted_rho[i][i].real
        else:
            prob_lst[i] = 0
    prob_lst = prob_lst / np.sum(prob_lst)
    measurement_res = np.random.choice(np.arange(0, dim), p=prob_lst)

    # calculate the post-measurement state
    rho_hat = np.outer(U[measurement_res,:].conj(),U[measurement_res,:])

    # call the map of channel M^{-1}
    rho_hat = H_map(rho_hat, V, X_mat, X_inv)
    return rho_hat

# ensemble 2: correct global shadow protocol, use random unitary
def Original_Global_Shadow(rho):
    dim = len(rho)
    U = np.array(random_unitary(dim).to_matrix())

    evoluted_rho = np.dot(U, np.dot(rho, np.conj(U).T))
    prob_lst = [0 for i in range(dim)]
    for i in range(dim):
        if abs(evoluted_rho[i][i]) > 10**(tol):
            prob_lst[i] = evoluted_rho[i][i].real
        else:
            prob_lst[i] = 0
    prob_lst = prob_lst / sum(prob_lst)
    measurement_res = np.random.choice(np.arange(0, dim), p=prob_lst)

    rho_hat = np.outer(U[measurement_res,:].conj(),U[measurement_res,:])
    rho_hat = (dim+1)*rho_hat - np.identity(dim)
    return rho_hat


In [ ]:
def Shadow_estimator(rho, obs, obs_type, M, ensemble, l, V, X_mat, X_inv, t_min, t_max):
    res = 0
    # Fidelity / Pauli Operator
    # CHANGE: whether to show progress bar for M
    M_tqdm = range(M)
    if (obs_type==0) or (obs_type==1):
        if ensemble == 0:
            for i in M_tqdm:
                rho_hat = Wrong_Global_Shadow(rho, l, V, t_min, t_max)
                res += np.trace(np.dot(obs, rho_hat))/M
        elif ensemble == 1:
            for i in M_tqdm:
                rho_hat = Hamiltonian_Shadow(rho, l, V, X_mat, X_inv, t_min, t_max)
                res += np.trace(np.dot(obs, rho_hat))/M
        elif ensemble == 2:
            for i in M_tqdm:
                rho_hat = Original_Global_Shadow(rho)
                res += np.trace(np.dot(obs, rho_hat))/M
    # Trace distance
    elif obs_type==2:
        if ensemble == 0:
            for i in M_tqdm:
                res += Wrong_Global_Shadow(rho, l, V, t_min, t_max)/M
        elif ensemble == 1:
            for i in M_tqdm:
                res += Hamiltonian_Shadow(rho, l, V, X_mat, X_inv, t_min, t_max)/M
        elif ensemble == 2:
            for i in M_tqdm:
                res += Original_Global_Shadow(rho)/M
    return res

# calculate average_bias and variance
# return 2 variable, 1st: average bias; 2nd: log(variance)
# Take average over ${times} estimators
def performance(rho, obs, obs_type, M, times, ensemble, l, V, X_mat, X_inv, t_min, t_max):
    var = 0
    bias = 0
    abs_bias = 0
    real_value = np.trace(np.dot(obs, rho))

    times_tqdm = tqdm(range(times),leave=False)
    for i in times_tqdm:
        estimator = Shadow_estimator(
            rho, obs, obs_type, M, ensemble, l, V, X_mat, X_inv, t_min, t_max)
        
        # Fidelity / Pauli Operator
        if (obs_type==0) or (obs_type==1):
            term = (estimator - real_value).real
        # Trace distance
        elif obs_type==2:
            term = (0.5*np.trace(sqrtm(np.dot((rho-estimator).conj().T,rho-estimator)))).real

        var += (term**2)/times
        bias += term/times
        abs_bias += np.abs(term)/times
    return bias, abs_bias, var


In [ ]:
# CHANGE: ways to put in H
def Get_lvx(h_group_num, theta_table, n):
    dim = 2**n
    l_lst = -1
    V_lst = []
    X_mat_lst = []
    X_inv_lst = []

    for i in range(h_group_num):
        P = random_hermitian(dim).to_matrix()

        V_lst.append([])
        X_mat_lst.append([])
        X_inv_lst.append([])
     
        for theta in theta_table:
            V = expm(1j*theta*P)
            X_mat, X_inv = Direct_calculate_X(V)
            
            V_lst[i].append(V)
            X_mat_lst[i].append(X_mat)
            X_inv_lst[i].append(X_inv)
    
    return l_lst,V_lst,X_mat_lst,X_inv_lst

In [ ]:
def Get_rho(n):
    # # 随机纯态
    # U0 = random_unitary(2**n).to_matrix()
    # rho0 = np.diag([1]+[0 for i in range(2**n-1)])
    # rho = np.dot(U0, np.dot(rho0, U0.conj().T))

    # GHZ态
    dim = 2**n
    rho = np.zeros((dim,dim))
    rho[0,0] = 0.5
    rho[0,dim-1] = 0.5
    rho[dim-1,0] = 0.5
    rho[dim-1,dim-1] = 0.5

    return rho

# CHANGE: need changing when measure Hamiltonian
def Get_obs(obs_type,rho,V):
    dim = len(rho)
    n = int(math.log(dim, 2))
    
    if obs_type==0: # Fidelity
        obs = rho
    
    elif obs_type==1: # Pauli
        z = [0]*n
        x = [1]*n
        obs = Pauli((z,x,0)).to_matrix()
    
    elif obs_type==2: # trace distance
        obs=np.zeros((2**n,2**n))
    
    elif obs_type==3: # Hamiltonian
        L = np.diag([np.random.rand() for _ in range(dim)])
        V_dag = V.conj().T
        H = np.dot(V,np.dot(L,V_dag))
        
        tr = np.trace(np.dot(H,H))
        tr = np.sqrt(tr.real)
        obs = H / tr

    return obs

In [ ]:
def create(path):
    if not os.path.exists(path):
        os.makedirs(path)

### Start from here

In [ ]:
M = 1000
n = 4
h_group_num = 10
# name = "theta_GHZ_fidelity&hamiltonian"
name = "theta_GHZ_median_fidelity&hamiltonian"

tol = -20

theta_table = [0.002,0.004,0.006,0.008,0.01,0.02,0.04,0.06,0.08,0.1,0.15,0.2,0.25,0.3,0.35]
theta_num = len(theta_table)

# 测的observable: obs1(Pauli), obs3(Hamiltonian)
obs_table = [1,3]
obs_num = len(obs_table)

ens_table = ['Ideal','Approx','Global']
ens_num = len(ens_table)
# address for saved data
addr = "./store/record_"+ name +".npy"
create("./store/")

In [ ]:
# CHANGE: arrangement of record
# record[ens_idx][obs_idx][theta_idx][P_idx]
record = np.zeros((ens_num,obs_num,theta_num,h_group_num))

In [ ]:
for obs_idx in range(len(obs_table)):
    obs_type = obs_table[obs_idx]
    
    print('n=',n,', M=', M, ', H_group_num=',h_group_num)
    print('--------------------------------')
    if obs_type==0:
        print('**** Fidelity test:')
    elif obs_type==1:
        print('**** Pauli test:')
    elif obs_type==2:
        print('**** trace distance test:')
    elif obs_type==3:
        print('**** Hamiltonian observable test:')

    # Get state rho
    rho = Get_rho(n)

    # Get Hamiltonian H and then calculate l,V,X,X_inv
    _,V_lst,X_mat_lst,X_inv_lst = Get_lvx(h_group_num,theta_table,n)

    # Get observable obs, a list in case we measure Hamiltonian
    obs_lst = []
    for P_idx in range(h_group_num):
        obs_lst.append([])
        for theta_idx in range(theta_num):
            obs = Get_obs(obs_type,rho,V_lst[P_idx][theta_idx])
            obs_lst[P_idx].append(obs)

    # CHANGE: different loop and different index
    # select different t intervals
    for theta_idx in range(theta_num):
        theta = theta_table[theta_idx]
        print('---------')
        print('** theta=',theta)
        print('')

        res = 0
        # h_tqdm = tqdm(range(h_group_num),leave=False)
        h_tqdm = range(h_group_num)
        
        # Ideal
        ens_idx = 0
        res_lst = [0]*h_group_num
        for P_idx in h_tqdm:
            res = Var_ideal(obs_lst[P_idx][theta_idx],rho,V_lst[P_idx][theta_idx],X_mat_lst[P_idx][theta_idx],X_inv_lst[P_idx][theta_idx],M)
            record[ens_idx][obs_idx][theta_idx][P_idx] = res
            res_lst[P_idx] = res
        res_median = np.median(res_lst)
        print('* Exact Ideal of Hamiltonian shadow: variance of Hamiltonian shadow=',res_median)
        # Approx
        ens_idx = 1
        res_lst = [0]*h_group_num
        for P_idx in h_tqdm:
            res = Var_approx(obs_lst[P_idx][theta_idx],V_lst[P_idx][theta_idx],X_mat_lst[P_idx][theta_idx],M)
            record[ens_idx][obs_idx][theta_idx][P_idx] = res
            res_lst[P_idx] = res
        res_median = np.median(res_lst)
        print('* Approximated variance of Hamiltonian shadow=',res_median)
        # global
        ens_idx = 2
        res_lst = [0]*h_group_num
        for P_idx in h_tqdm:
            res = Var_global(obs_lst[P_idx][theta_idx],rho,M)
            record[ens_idx][obs_idx][theta_idx][P_idx] = res
            res_lst[P_idx] = res
        res_median = np.median(res_lst)
        print('* Global shadow: variance=',res_median)
        print('')

    np.save(addr, record)